In [68]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
data = pd.read_pickle('data.pandas')

In [70]:
data.head()

,address,body,date_sent,readable_date,service_center,read,type,id,ClusterId,DistrictCity,...,DATE,COUNT,PSU_ID,URB_RUR,REGION,DISTRICT,LONG,LAT,dateCollectionStartDays,_merge
0,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 10:11:54,2016-10-19 10:11:54,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
1,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 11:11:57,2016-10-19 11:11:57,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
2,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 12:11:57,2016-10-19 12:11:57,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
3,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 13:11:58,2016-10-19 13:11:58,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
4,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 14:12:28,2016-10-19 14:12:28,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both


In [71]:
data['dateSentDays'] = data['date_sent'].dt.date
data['dateCollectionStartDays'] = pd.to_datetime(data['DateCollectionStart'], infer_datetime_format=True).dt.date
data = data[data['dateSentDays'] >= data['dateCollectionStartDays']]

In [72]:
bodyNoId = pd.Series(data['body'].str.strip().str.replace('\d+', '').str.lower().str.replace('id', '').str.replace('  ', ' '))
bodyNoId = bodyNoId.str.replace('.', '').str.replace('  ', ' ').str.replace('_ ', '').str.replace('l ', '')
bodyNoId.value_counts()

power failure is running | monitoring power ok                   41884
power failure detected                                           38433
power back detected                                              37033
power failure is running | monitoring power failure              11585
power back                                                         226
power failure                                                      224
power back detected                                                135
test sms from power failure monitor:power failure detected          54
test sms from power failure monitor:power back detected             38
test sms from power failure monitor:power failure is running         8
test sms from power failure monitor:power back detec                 2
Name: body, dtype: int64

In [225]:
power = data[['id', 'date_sent']].copy()
power['body'] = bodyNoId.copy()
power['powerBack'] = bodyNoId.str.contains('back') & ~(bodyNoId.str.contains('monitor'))
power['powerOff'] = ( (bodyNoId.str.contains('failure detected')) | \
    (bodyNoId.str.match('power failure')) ) & \
    ~(bodyNoId.str.contains('monitor'))

In [226]:
power[power.powerBack | power.powerOff].head()

,id,date_sent,body,powerBack,powerOff
54,1301.0,2016-11-08 17:52:45,power back detected,True,False
55,1301.0,2016-11-08 21:12:36,power failure detected,False,True
56,1301.0,2016-11-08 21:19:23,power failure detected,False,True
57,1301.0,2016-11-08 21:19:26,power back detected,True,False
58,1301.0,2016-11-09 17:29:17,power failure detected,False,True


In [170]:
data[(data['id'] == 1301) & \
     ((data['date_sent'].dt.date == pd.to_datetime('2016-11-14').date()) | \
      (data['date_sent'].dt.date == pd.to_datetime('2016-11-15').date()))]

,address,body,date_sent,readable_date,service_center,read,type,id,ClusterId,DistrictCity,...,COUNT,PSU_ID,URB_RUR,REGION,DISTRICT,LONG,LAT,dateCollectionStartDays,_merge,dateSentDays
81,+992935710546,Power Back detected. id 1301,2016-11-14 05:03:19,2016-11-14 05:03:19,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
82,+992935710546,Power Failure detected. id 1301,2016-11-14 08:05:04,2016-11-14 08:05:04,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
83,+992935710546,Power Back detected. id 1301,2016-11-14 17:59:32,2016-11-14 17:59:32,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
84,+992935710546,Power Failure detected. id 1301,2016-11-14 20:39:27,2016-11-14 20:39:27,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
85,+992935710546,Power Back detected. id 1301,2016-11-14 20:39:59,2016-11-14 20:39:59,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
86,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-11-14 21:07:46,2016-11-14 21:07:46,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
87,+992935710546,Power Failure detected. id 1301,2016-11-14 22:03:23,2016-11-14 22:03:23,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
88,+992935710546,Power Failure detected. id 1301,2016-11-15 08:04:08,2016-11-15 08:04:08,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-15
89,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-11-15 09:07:27,2016-11-15 09:07:27,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-15
90,+992935710546,Power Back detected. id 1301,2016-11-15 17:57:43,2016-11-15 17:57:43,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-15


In [ ]:
"""
In case of more than one off & back per hour, we keep the last one 
  Extract date and hour and make that index, then keep max of date_sent
(since by definition, all other changes inside the hour are only minutes long)
Keep only one per hour.
To impute missing, we need to now if hours are on or off So we can only work with these messages.
Fill index again to complete the hours and merge these indicators
Start by assuming status on, then go through hours and turn indicator off if power off and on if on.
"""

In [227]:
power = power[power.powerBack | power.powerOff]
power['date_sent_hour'] = power['date_sent']
power.set_index('date_sent_hour', drop=True, inplace=True)
power.index = power.index.floor('H') # But doing this can create two records in the same hour
power.reset_index(inplace=True)
powerNoDup = pd.DataFrame(power[['id', 'date_sent_hour', 'date_sent']].groupby(['id', 'date_sent_hour']).max(), copy=True)
powerNoDup.reset_index(inplace=True)

In [228]:
power = power.drop(['date_sent_hour', 'body'], 1).merge(powerNoDup.drop('date_sent_hour', 1), 
                                           left_on=['id', 'date_sent'], right_on=['id', 'date_sent'], how='right')

In [229]:
power.set_index('date_sent', drop=True, inplace=True)
power.index = power.index.floor('H') # But doing this can create two records in the same hour
power.reset_index(inplace=True)
power.head()

,date_sent,id,powerBack,powerOff
0,2016-11-08 17:00:00,1301.0,True,False
1,2016-11-08 21:00:00,1301.0,True,False
2,2016-11-09 17:00:00,1301.0,True,False
3,2016-11-09 22:00:00,1301.0,False,True
4,2016-11-10 08:00:00,1301.0,False,True


In [230]:
power = power.drop('powerOff', 1)
power.to_pickle('power.pandas')

In [231]:
missingHours = pd.read_pickle('missingHours.pandas')

In [232]:
merged = power.merge(missingHours, left_on=['id', 'date_sent'], right_on=['id', 'date_sent'], how='right', indicator=True)

In [233]:
merged._merge.value_counts()

right_only    656854
both           46485
left_only          0
Name: _merge, dtype: int64

In [ ]:
"""
Both are hours where there was a power on or power off message
Right only, are the ones we must fill
"""

In [234]:
# Must sort in order to fill
merged.set_index(['id', 'date_sent'], inplace=True)
merged.sort_index(inplace=True)

In [235]:
merged.to_pickle('merged.pandas')